# Token House Votes

In [ ]:
import pandas as pd

In [2]:
df_Top1000Delegates = pd.read_csv("./Top1000Delegates.csv")
df_vote_cast = pd.read_csv("./vote_cast.csv")
df_proposals = pd.read_csv("./proposals.csv")

In [ ]:
df_proposals_votes = pd.read_csv("./raw_data/2024-06-28_optimism_proposals_votes.csv")

In [15]:
df_onchain = df_proposals_votes.query('vote_type == "ON_CHAIN"').reset_index(drop=True)

# Step 1: Sort the DataFrame by 'proposal_id' and 'voting_power' in descending order
df_sorted = df_onchain.sort_values(by=['proposal_id', 'voting_power'], ascending=[True, False])

# Step 2: Group by 'proposal_id' and select the top 200 for each group
df_top_200 = df_sorted.groupby('proposal_id').head(200).reset_index(drop=True)

df_top_200['reason'] = df_top_200['reason'].fillna('NA')

In [21]:
df_top_200.to_csv("./proposals_votes.csv")

# Citizen House Votes

In [61]:
df_citizen_snapshot_proposal = pd.read_json('citizen_snapshot_proposals.json')
df_citizen_snapshot_proposal=df_citizen_snapshot_proposal.rename(columns={'id':'proposal_id'})

In [63]:
import requests

df_citizen_snapshot_proposal_votes = pd.DataFrame()

for id in df_citizen_snapshot_proposal.proposal_id.to_list():
    url = f"https://hub.snapshot.org/graphql?operationName=Votes&query=query%20Votes%20%7B%0A%20%20votes%20(%0A%20%20%20%20first%3A%201000%0A%20%20%20%20skip%3A%200%0A%20%20%20%20where%3A%20%7B%0A%20%20%20%20%20%20proposal%3A%20%22{id}%22%0A%20%20%20%20%7D%0A%20%20%20%20orderBy%3A%20%22created%22%2C%0A%20%20%20%20orderDirection%3A%20desc%0A%20%20)%20%7B%0A%20%20%20%20id%0A%20%20%20%20voter%0A%20%20%20%20created%0A%20%20%20%20proposal%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%7D%0A%20%20%20%20choice%0A%20%20%20%20space%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A"
    response = requests.get(url)
    temp = response.json()['data']['votes']
    for t in range(len(temp)):
        temp[t]['proposal_id'] = temp[t]['proposal']['id']
        temp[t]['space_id'] = temp[t]['space']['id']
    temp_df = pd.DataFrame(temp)
    df_citizen_snapshot_proposal_votes = pd.concat([df_citizen_snapshot_proposal_votes,temp_df])

In [65]:

df_citizen_snapshot_proposal_votes = pd.merge(df_citizen_snapshot_proposal_votes,df_citizen_snapshot_proposal,how="left",on='proposal_id')

In [66]:
df_citizen_snapshot_proposal_votes.to_csv('./citizen_snapshot_proposal_votes.csv')

In [41]:
df_citizen_snapshot_proposal.id

0     0xff9fcfd224f44cdcb6e8118cc5909e1376044d4e2640...
1     0x14336dfcb086279e47ef8fffbd6282984d392f1b9eaf...
2     0xcbf088e409279ca2e1dbe78f513067dd20caa4d2d248...
3     0x21f7126c1636cecdcf7522eadbf6e1b20ca22a2230fa...
4     0xe2d86afceea73f2e2734550bec73af395b78826413fb...
5     0x3c45ddb959dcab99231e29cafad668e58c99040b5c64...
6     0x2bc6565053b73813c6b0a001c8c07eb5656234b4d8ba...
7     0x8c157e9d2718a572626ad8443c494fae5986bd467dcc...
8     0x9e85524badfcec19856bfc51cfefc78cb5f36a9b77ec...
9     0x6efd678a37dd547f40956fb613047ffc077c234b7723...
10    0x1552e2685f2f4560d9af1f69cba056d5a3e5b78ee356...
Name: id, dtype: object